In [1]:
cd /home

/home


In [2]:
import matplotlib.pyplot as plt
import mlflow

In [5]:
runs_df = mlflow.search_runs(experiment_ids=['852161655664768094'], 
                              search_all_experiments=True)
runs_df = runs_df[runs_df.status == 'FINISHED']

runs_df = runs_df[['tags.downstream_data', 'tags.arch', 'tags.m_alpha', 
                   'tags.probed', 'tags.rs', 'tags.same', 'tags.spc', 
                   'tags.fold', 'metrics.test_f1']]
runs_df['tags.fold'] = runs_df['tags.fold'].astype(float)
runs_df['metrics.test_f1'] = runs_df['metrics.test_f1'].astype(float)

In [6]:
runs_df.groupby(['tags.downstream_data', 'tags.arch', 'tags.m_alpha', 
                 'tags.probed', 'tags.rs', 'tags.same', 'tags.spc']).agg({'metrics.test_f1': ['mean', 'std']})

metrics.test_f1  \
                                                                                              mean   
tags.downstream_data tags.arch tags.m_alpha tags.probed tags.rs tags.same tags.spc                   
atlas                base      1.0          0.5         0.2     0.8       100             0.565016   
                                                                          20              0.585054   
                                                                          500             0.651385   
macho                base      1.0          0.5         0.2     0.8       100             0.575650   
                                                                          20              0.233617   
                                                                          500             0.618645   

                                                                                              
                                                                                         std  
tags.downstream_data tags.arch tags.m_alpha tags.probed tags.rs tags.same tags.spc            
atlas                base      1.0          0.5         0.2     0.8       100       0.165939  
                                                                          20        0.076211  
                                                                          500       0.052051  
macho                base      1.0          0.5         0.2     0.8       100       0.039289  
                                                                          20        0.274173  
                                                                          500       0.003478